In [9]:
%pip install tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 600.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\tqdm.exe' -> 'c:\\Python312\\Scripts\\tqdm.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
import time
from tqdm import tqdm

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Variables List + Header API

In [3]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
my_game_name = os.getenv("my_game_name")
my_tag_line = os.getenv("my_tag_line")

In [4]:
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

In [7]:
apex_tiers = ['challengerleagues', 'grandmasterleagues', 'masterleagues']
#divisions = ['DIAMOND', "EMERALD"]
divisions = ['DIAMOND']
tiers = ["I", "II", "III", "IV"]
#tiers = ["I"]

CURRENT_PATCH = "15.3"

In [ ]:
def get_summoner_ids_from_api(api_url, req_type):
    summoner_ids = []
    api_url = api_url #dont think this needs to be here lol
    try:
        resp = requests.get(api_url, headers=headers )  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")   
    response = resp.json()
    if req_type == "apex":
        entries = response['entries']
        for entry in entries:
            summoner_ids.append(entry['summonerId'])
    if req_type == "regular":
        for entry in response:
            summoner_ids.append(entry['summonerId'])
    return summoner_ids


def get_sums_for_apex_leagues():
    apex_summoner_ids = []
    for apex_league in apex_tiers:
        print(apex_league)
        api_url = f"https://na1.api.riotgames.com/lol/league/v4/{apex_league}/by-queue/RANKED_SOLO_5x5"
        summoner_ids = get_summoner_ids_from_api(api_url, "apex")
        apex_summoner_ids += summoner_ids
    return apex_summoner_ids

def get_sums_for_reg_divisions(start_time):
    inner_start_time = start_time
    total_loops = 3 #starts at 3 since there should be 3 apex tier calls before this
    reg_division_ids = []
    for division in divisions:
        print(division)
        for tier in tiers:
            print(tier)
            page = 1
            more_pages=True
            while more_pages:
                if total_loops == 100: #if we have made 100 requests
                    dif = time.time()-inner_start_time
                    if dif< 120: #if it has not been 120 seconds
                        print(f'sleeping for {dif}')
                        time.sleep(dif)
                        inner_start_time = time.time() #reset start time
                        total_loops = 0 #reset number of loops as well
                api_url = f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page={page}'
                output_list = get_summoner_ids_from_api(api_url, "regular")
                if output_list:
                    print(page)
                    page += 1
                    reg_division_ids += output_list
                else:
                    more_pages = False
                total_loops += 1 #keeping track of number of api calls for rate limiting purposes
    return reg_division_ids

def get_all_wanted_sums():
    all_sum_ids = []
    start = time.time()
    all_sum_ids += get_sums_for_apex_leagues()
    all_sum_ids += get_sums_for_reg_divisions(start)
    return all_sum_ids


#could later make this a dataframe the whole time, and add the rank information along with the summoner ID
# schema = {'summonerId': 'str', 'Division': 'str', 'Tier': 'str'}
# df = pd.DataFrame(columns=schema.keys()).astype(schema)

all_wanted_sum_ids = get_all_wanted_sums()
#print(all_wanted_sum_ids)
print(len(all_wanted_sum_ids))
all_wanted_sum_ids = list(set(all_wanted_sum_ids))
print(len(all_wanted_sum_ids)) #see if it dropped any duplicates

#took ~4 mins to do diamond +


In [ ]:
sums_df = pd.DataFrame({"summonerId": all_wanted_sum_ids})
sums_df

In [ ]:
sums_df.to_csv("all_wanted_summoner_ids.csv")

# Function to read csv above and populate it (and the dataframe) with puuids in a new column

In [5]:
def get_puuid(csv_only_summ_ids):
    # Assigning csv file here as a placeholder, can just include it in the argument when using this fucntion
    df_only_summ_ids = pd.read_csv(csv_only_summ_ids, index_col=False)
    # Loop to iterate through each row
    for index, row in df_only_summ_ids.iterrows():
            summoner_id = row["summonerId"]
            # Variable to insert summ_id in API url that fetches account data
            api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            try:
                resp = requests.get(api_url, headers=headers)  #Insert API link here
                resp.raise_for_status()
                summoner_info = resp.json()
                # Create a column in the data frame called puuids and populate it with the puuid for each summoner id
                df_only_summ_ids.at[index, "puuid"] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
            # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
            except requests.RequestException as e:
                print(f"Error fetching puuid for {summoner_id}: {e}")
    # Update csv file with the new column
    df_only_summ_ids.to_csv(csv_only_summ_ids, index = False)
# Calling function to add new info to existing csv instead of creating new one (using test csv from above in this case)

In [ ]:
get_puuid("all_wanted_summoner_ids.csv")

# Function to get match ids using puuids from Dataframe

In [6]:
def get_match_ids(csv_summids_and_puuids, current_patch):
    # Read csv file with summoner ids and puuids 
    df_summoners = pd.read_csv(csv_summids_and_puuids)
    # Create a new DataFrame to store match data
    df_all_matches = pd.DataFrame(columns = ["summonerId", "puuid", "matchId", "matchData"])
    # Variable for number of matches, will be defaulted to max (100) in actual code, small number used for testing
    number_of_matches = 5
    # Iterate through each row in df_summoners
    for _, row in df_summoners.iterrows():
        summoner_id = row["summonerId"]
        puuid = row["puuid"]
        # Fetch match ids for each puuid
        match_history_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={number_of_matches}"

        try:
            resp = requests.get(match_history_api_url, headers=headers)
            resp.raise_for_status()
            match_ids = resp.json()
            # Iterate through each match id and fetch match data
            for match_id in match_ids:
                match_data_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
                response = requests.get(match_data_api_url, headers=headers)
                match_data = response.json()
                # Variable to check for patch/version
                game_patch = (match_data["info"]["gameVersion"].split("."))[0] + "." + (match_data["info"]["gameVersion"].split("."))[1]
                if game_patch == current_patch:
                    # Store match data in JSON file along with summoner id and puuid
                    df_mini = pd.DataFrame({
                        "summonerId" : [summoner_id],
                        "puuid" : [puuid],
                        "matchId" : [match_id],
                        "matchData": [json.dumps(match_data)]
                        })
                    # Append to main DataFrame
                    df_all_matches = pd.concat([df_all_matches, df_mini], ignore_index = True)
                else:
                    continue
        # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
        except requests.RequestException as e:
            print(f"Error fetching matches for {puuid}: {e}")
    df_all_matches = df_all_matches.drop_duplicates("matchId")
    # Save the expanded DataFrame
    df_all_matches.to_csv("all_match_data.csv", index = False)
    print(df_all_matches.nunique())
    return df_all_matches

In [7]:
# Running get_match function, argument 1 = csv file, argument 2 = current patch
get_match_ids("all_wanted_summoner_ids.csv", CURRENT_PATCH)

summonerId     4
puuid          4
matchId       20
matchData     20
dtype: int64


,summonerId,puuid,matchId,matchData
0,QkSyD-ZIMQz1G7X7lLe5547whcdvwA0LEQZwId022S0_eq...,IfWAMXpMUQ4PVOqJRAQc0U2AFVe8tXdbwDxcSWU7ppSn31...,NA1_5229648468,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
1,QkSyD-ZIMQz1G7X7lLe5547whcdvwA0LEQZwId022S0_eq...,IfWAMXpMUQ4PVOqJRAQc0U2AFVe8tXdbwDxcSWU7ppSn31...,NA1_5224738854,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
2,QkSyD-ZIMQz1G7X7lLe5547whcdvwA0LEQZwId022S0_eq...,IfWAMXpMUQ4PVOqJRAQc0U2AFVe8tXdbwDxcSWU7ppSn31...,NA1_5224663404,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
3,QkSyD-ZIMQz1G7X7lLe5547whcdvwA0LEQZwId022S0_eq...,IfWAMXpMUQ4PVOqJRAQc0U2AFVe8tXdbwDxcSWU7ppSn31...,NA1_5224632058,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
4,QkSyD-ZIMQz1G7X7lLe5547whcdvwA0LEQZwId022S0_eq...,IfWAMXpMUQ4PVOqJRAQc0U2AFVe8tXdbwDxcSWU7ppSn31...,NA1_5223808666,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
5,PchKnRxFWUOUZ-0xCghC7wsEX9SPnOHX1NfFzHdjB_dvonE,PLwl77quh4HM-JxQyq8XPrm2T7upfQSwgGc7Tm06S3zwS5...,NA1_5226861599,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
6,PchKnRxFWUOUZ-0xCghC7wsEX9SPnOHX1NfFzHdjB_dvonE,PLwl77quh4HM-JxQyq8XPrm2T7upfQSwgGc7Tm06S3zwS5...,NA1_5226826880,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
7,PchKnRxFWUOUZ-0xCghC7wsEX9SPnOHX1NfFzHdjB_dvonE,PLwl77quh4HM-JxQyq8XPrm2T7upfQSwgGc7Tm06S3zwS5...,NA1_5226791216,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
8,PchKnRxFWUOUZ-0xCghC7wsEX9SPnOHX1NfFzHdjB_dvonE,PLwl77quh4HM-JxQyq8XPrm2T7upfQSwgGc7Tm06S3zwS5...,NA1_5226766682,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
9,PchKnRxFWUOUZ-0xCghC7wsEX9SPnOHX1NfFzHdjB_dvonE,PLwl77quh4HM-JxQyq8XPrm2T7upfQSwgGc7Tm06S3zwS5...,NA1_5226500664,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."


Aggregate all data at the champion level:
Created 3 functions:
1. create_match_df(): 
    Takes a dataframe as input (for now it takes a csv file but can be easily changed to a df if needed). It then accesses the matchData column of that df and calls the create_df_participants() and create_df_teamdata() functions using the df it took from input (assigned to df_all_matches) as the argument for both functions. 
    - Next step would be to aggregate the data obtained from both helper functions at the champion level

2. create_df_participants():
    Takes the "matchData" column of df inputted to create_match_df() as input. It then extracts the value from the participant key (inside the info dictionary) and creates a new df where each participant in each game is a row and the columns are the data points from the matchdata API endpoint. This function flattens out the entire dictionary (so the keys from sub-dictionaries like "challenges" are columns). This also includes matchIds.

3. create_df_teamdata():  
    Takes the "matchData" column of df inputted to create_match_df() as input. It then extracts the value from the teams key (inside the info dictionary) and creates a new df where each team in each game is a row and the columns are the data points from the matchdata API endpoint (bans, objectives, etc.).
    IMPORTANT: The bans information is still nested in a list of dictionaries, where each dictionary is a ban.



In [12]:
# Function to create a list with all match data
def create_match_df(dataframe_csv: str) -> list[dict]:
    df_all_matches = pd.read_csv(dataframe_csv)
    df_all_matches["matchData"] = df_all_matches["matchData"].apply(json.loads)
    df_participants = create_df_participants(df_all_matches)
    df_teams = create_df_teamdata(df_all_matches)
    return df_participants, df_teams  
    
# Create a function to create a DataFrame from the participants column (i.e. a list of dictionaries)
def create_df_participants(df_all_matches: pd.DataFrame) -> pd.DataFrame:
    df_all_matches["participants"] = df_all_matches["matchData"].apply(lambda m: m["info"]["participants"])
    df_exploded = df_all_matches.explode("participants").reset_index(drop=True)
    df_participants = pd.json_normalize(df_exploded["participants"], sep = "_")
    df_participants["matchId"] = df_exploded["matchData"].apply(lambda m: m["metadata"]["matchId"])
    return df_participants

# Create a function to create a DataFrame with data from each team (each row is a team), like bans and objectives
def create_df_teamdata(df_all_matches: pd.DataFrame) -> pd.DataFrame:
    df_all_matches["teams"] = df_all_matches["matchData"].apply(lambda m: m["info"]["teams"])
    df_exploded = df_all_matches.explode("teams").reset_index(drop=True)
    df_teams = pd.json_normalize(df_exploded["teams"], sep = "_")
    df_teams["matchId"] = df_exploded["matchData"].apply(lambda m: m["metadata"]["matchId"])
    return df_teams

In [9]:
df_participants, df_teams = create_match_df("all_match_data.csv")

In [21]:
df_participants.dtypes

PlayerScore0                                              int64
PlayerScore1                                              int64
PlayerScore10                                             int64
PlayerScore11                                             int64
PlayerScore2                                              int64
PlayerScore3                                              int64
PlayerScore4                                              int64
PlayerScore5                                              int64
PlayerScore6                                              int64
PlayerScore7                                              int64
PlayerScore8                                              int64
PlayerScore9                                              int64
allInPings                                                int64
assistMePings                                             int64
assists                                                   int64
baronKills                              

In [16]:
df_teams.dtypes

bans                           object
teamId                          int64
win                              bool
objectives_atakhan_first         bool
objectives_atakhan_kills        int64
objectives_baron_first           bool
objectives_baron_kills          int64
objectives_champion_first        bool
objectives_champion_kills       int64
objectives_dragon_first          bool
objectives_dragon_kills         int64
objectives_horde_first           bool
objectives_horde_kills          int64
objectives_inhibitor_first       bool
objectives_inhibitor_kills      int64
objectives_riftHerald_first      bool
objectives_riftHerald_kills     int64
objectives_tower_first           bool
objectives_tower_kills          int64
matchId                        object
dtype: object

In [ ]:
def aggregate_champion_data(df_participants: pd.DataFrame, df_teams: pd.DataFrame) -> pd.DataFrame:
    # Create a teamId column in participants based on participantId (1-5 = 100, 6-10 = 200)
    df_participants['teamId'] = df_participants['participantId'].apply(lambda x: 100 if x <= 5 else 200)
    
    # Merge participants with their team data
    merged_df = df_participants.merge(
        df_teams,
        on=['matchId', 'teamId'],
        how='left'
    )
    
    # Aggregate by champion
    champion_stats = merged_df.groupby(['championId', 'championName']).agg({
        # Core stats
        'kills': 'mean',
        'deaths': 'mean',
        'assists': 'mean',
        'matchId': 'count', # Count number of games played per champion
        
        # Combat stats
        'totalDamageDealtToChampions': 'mean',
        'totalDamageTaken': 'mean',
        'magicDamageDealtToChampions': 'mean',
        'physicalDamageDealtToChampions': 'mean',
        'trueDamageDealtToChampions': 'mean',
        'largestCriticalStrike': 'mean',
        'timeCCingOthers': 'mean',
        'totalHealsOnTeammates': 'mean',
        'totalDamageShieldedOnTeammates': 'mean',
        
        # Economy stats
        'goldEarned': 'mean',
        'goldSpent': 'mean',
        'totalMinionsKilled': 'mean',
        'neutralMinionsKilled': 'mean',
        
        # Vision stats
        'visionScore': 'mean',
        'wardsPlaced': 'mean',
        'wardsKilled': 'mean',
        'visionWardsBoughtInGame': 'mean',
        
        # Team objectives
        'objectives_baron_kills': 'mean',
        'objectives_champion_kills': 'mean',
        'objectives_dragon_kills': 'mean',
        'objectives_inhibitor_kills': 'mean',
        'objectives_riftHerald_kills': 'mean',
        'objectives_tower_kills': 'mean',
        
        # First objective rates
        'objectives_baron_first': 'mean',
        'objectives_dragon_first': 'mean',
        'objectives_inhibitor_first': 'mean',
        'objectives_riftHerald_first': 'mean',
        'objectives_tower_first': 'mean',
        
        # Game length related
        'timePlayed': 'mean',
        'longestTimeSpentLiving': 'mean',
        
        # Multikill stats
        'doubleKills': 'mean',
        'tripleKills': 'mean',
        'quadraKills': 'mean',
        'pentaKills': 'mean',
        
        # Combat achievements
        'firstBloodKill': 'mean',
        'firstTowerKill': 'mean',
        'turretKills': 'mean',
        'inhibitorKills': 'mean',
        
    }).round(2)
    
    # Rename columns for clarity
    champion_stats = champion_stats.rename(columns={
        'matchId': 'games_played', # Rename the count column to a more descriptive name
        'objectives_baron_first': 'first_baron_rate',
        'objectives_dragon_first': 'first_dragon_rate',
        'objectives_inhibitor_first': 'first_inhibitor_rate',
        'objectives_riftHerald_first': 'first_herald_rate',
        'objectives_tower_first': 'first_tower_rate',
        'firstBloodKill': 'first_blood_rate',
        'firstTowerKill': 'first_tower_kill_rate'
    })
    
    # Calculate derived stats
    champion_stats['kda'] = ((champion_stats['kills'] + champion_stats['assists']) / 
                            champion_stats['deaths']).round(2)
    champion_stats['cs_per_minute'] = ((champion_stats['totalMinionsKilled'] + champion_stats['neutralMinionsKilled']) / 
                                      (champion_stats['timePlayed'] / 60)).round(2)
    champion_stats['gold_per_minute'] = (champion_stats['goldEarned'] / 
                                        (champion_stats['timePlayed'] / 60)).round(2)
    champion_stats['damage_per_minute'] = (champion_stats['totalDamageDealtToChampions'] / 
                                          (champion_stats['timePlayed'] / 60)).round(2)
    
    return champion_stats

# Create and display the aggregated data
champion_stats = aggregate_champion_data(df_participants, df_teams)
display(champion_stats.sort_values('games_played', ascending=False))

,,kills,deaths,assists,games_played,totalDamageDealtToChampions,totalDamageTaken,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,largestCriticalStrike,timeCCingOthers,totalHealsOnTeammates,totalDamageShieldedOnTeammates,goldEarned,goldSpent,totalMinionsKilled,neutralMinionsKilled,visionScore,wardsPlaced,wardsKilled,visionWardsBoughtInGame,objectives_baron_kills,objectives_champion_kills,objectives_dragon_kills,objectives_inhibitor_kills,objectives_riftHerald_kills,objectives_tower_kills,first_baron_rate,first_dragon_rate,first_inhibitor_rate,first_herald_rate,first_tower_rate,timePlayed,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,first_blood_rate,first_tower_kill_rate,turretKills,inhibitorKills,kda,cs_per_minute,gold_per_minute,damage_per_minute
championId,championName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
799,Ambessa,5.38,6.00,6.12,8,19112.38,23202.25,167.75,18184.38,759.62,0.00,10.62,0.00,0.00,10086.88,9437.50,136.75,31.62,17.75,7.75,2.25,1.62,0.12,24.12,1.88,0.50,0.62,5.12,0.12,0.50,0.38,0.62,0.38,1515.12,443.38,0.38,0.00,0.0,0.0,0.12,0.00,0.75,0.00,1.92,6.67,399.45,756.87
86,Garen,5.67,5.17,3.00,6,24990.83,35536.17,277.17,18638.17,6074.50,698.50,38.67,0.00,0.00,12982.50,12351.33,243.00,3.83,18.17,6.33,4.33,0.50,0.33,23.67,2.17,0.67,0.50,6.00,0.33,0.83,0.17,0.50,0.50,1796.00,341.17,0.33,0.00,0.0,0.0,0.00,0.00,2.00,0.33,1.68,8.25,433.71,834.88
110,Varus,6.17,4.00,7.17,6,19666.50,15685.00,3736.17,15762.83,167.17,0.00,23.67,0.00,0.00,11494.67,11000.83,200.33,2.50,14.83,9.67,2.33,1.17,0.17,29.17,0.67,0.67,0.83,6.17,0.17,0.00,0.33,0.83,0.67,1483.33,436.17,0.50,0.33,0.0,0.0,0.00,0.17,2.17,0.17,3.34,8.20,464.95,795.50
42,Corki,7.50,3.83,6.00,6,25068.17,21538.50,5275.00,16481.83,3310.00,432.67,1.33,0.00,0.00,13556.17,12845.50,246.17,11.67,21.67,9.00,3.83,1.50,0.00,27.00,1.33,0.83,0.83,5.33,0.00,0.33,0.33,0.83,0.33,1743.67,793.67,1.33,0.17,0.0,0.0,0.17,0.17,2.00,0.17,3.52,8.87,466.47,862.60
92,Riven,6.60,4.60,6.00,5,22803.80,24394.80,100.60,21501.00,1201.60,242.40,27.80,0.00,0.00,13122.20,12105.00,231.00,4.40,21.60,8.60,4.20,1.00,0.40,32.00,2.40,0.80,1.00,6.40,0.40,0.60,0.60,1.00,0.60,1855.80,613.40,1.00,0.00,0.0,0.0,0.00,0.20,1.20,0.00,2.74,7.61,424.25,737.27
64,LeeSin,7.00,4.80,8.60,5,17542.20,31700.80,2686.80,13993.40,860.80,185.00,19.60,0.00,1032.00,10626.20,9835.00,16.20,133.60,27.60,13.60,3.40,5.80,0.20,29.40,1.00,0.20,0.20,4.40,0.20,0.40,0.20,0.20,0.20,1544.20,637.80,0.60,0.20,0.0,0.0,0.20,0.00,0.80,0.00,3.25,5.82,412.88,681.60
111,Nautilus,4.80,6.80,11.80,5,11335.60,27003.60,5892.20,4466.20,976.00,7.00,65.80,52.20,370.60,8527.00,7825.00,28.40,0.00,64.40,30.20,8.00,5.00,0.40,31.20,1.40,1.00,0.60,7.20,0.40,0.40,0.60,0.60,1.00,1638.40,365.60,0.40,0.00,0.0,0.0,0.00,0.20,0.60,0.00,2.44,1.04,312.27,415.12
50,Swain,4.00,6.50,9.75,4,30144.25,46827.25,27250.75,2280.75,611.75,0.00,67.00,0.00,0.00,12488.75,11937.50,235.25,1.25,20.25,9.50,4.75,1.00,0.75,30.25,1.50,0.50,0.50,5.50,0.50,0.00,0.25,0.50,0.75,1994.75,636.00,0.25,0.00,0.0,0.0,0.00,0.25,0.75,0.00,2.12,7.11,375.65,906.71
62,MonkeyKing,6.00,4.00,11.25,4,16672.25,29547.75,1850.75,14028.25,792.50,90.50,14.50,0.00,0.00,10195.50,11452.50,36.00,115.50,20.75,6.75,3.50,3.25,0.25,30.75,1.50,0.25,0.00,4.75,0.25,0.50,0.25,0.00,0.75,1455.50,457.25,1.00,0.00,0.0,0.0,0.00,0.00,0.75,0.25,4.31,6.25,420.29,687.28
